In [64]:
from src.models.liquidity_distribution import LiquidityAnalyzer, Pool
from datetime import date
import math
import pandas as pd
import numpy as np
from src.data_processing.fetch_data import fetch_oku_liquidity, fetch_data_subgraph

In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:

POOLS = [
    (3, "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", "v3 USDC/ETH 0.3%", ),
    (3, "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", "v3 USDC/ETH 0.05%", ),
]

In [118]:
POOL_ADDRESS = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"

In [119]:
pool_query = """query get_pools($pool_id: ID!) {
  pools(where: {id: $pool_id}) {
    tick
    sqrtPrice
    liquidity
    feeTier
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""
variables = {'pool_id': POOL_ADDRESS}
API_KEY = "893d7471304c5edf436c8ba60781762c"
data_pool = fetch_data_subgraph(API_KEY, pool_query, variables, data_key='pools', first_n=100, batch_size=1000)[0]

Fetched 1 records


In [120]:
# data = fetch_oku_liquidity(pool_address=POOL_ADDRESS, block_number=20719648)

In [121]:
START_DATE = date(2024, 9, 1)
END_DATE = date(2024, 9, 2)

In [126]:
pool = Pool(pool_address=POOL_ADDRESS,
            token0="WETH",
            token1="USDC",
            decimals0=data_pool['token0']['decimals'],
            decimals1=data_pool['token1']['decimals'],
            fee_tier=data_pool['feeTier'],
            sqrt_price_x96=data_pool["sqrtPrice"])

analyzer = LiquidityAnalyzer(
    pool,
    start_date=START_DATE,
    end_date=END_DATE
)

Loading liquidity for block number 20652150
Loading liquidity for block number 20652292
Loading liquidity for block number 20652391
Loading liquidity for block number 20652488
Loading liquidity for block number 20652623
Loading liquidity for block number 20652645
Loading liquidity for block number 20652675
Loading liquidity for block number 20652715
Loading liquidity for block number 20652758
Loading liquidity for block number 20652820
Loading liquidity for block number 20652952
Loading liquidity for block number 20653060
Loading liquidity for block number 20653152
Loading liquidity for block number 20653243
Loading liquidity for block number 20653427
Loading liquidity for block number 20653570
Loading liquidity for block number 20653625
Loading liquidity for block number 20653633
Loading liquidity for block number 20653649
Loading liquidity for block number 20653660
Loading liquidity for block number 20653692
Loading liquidity for block number 20653753
Loading liquidity for block numb

In [128]:
pool.decimals0, pool.decimals1, pool.fee_tier, pool.tick_spacing

(6, 18, 500, 10)

In [117]:
pool.decimals0, pool.decimals1, pool.fee_tier, pool.tick_spacing

(6, 18, '3000', 60)

In [112]:
liquidity = pd.merge(analyzer.sampled_blocks,
              analyzer.df_liquidity,
              left_on='block_number',
              right_on='block')

In [113]:
def subselect_active_liquidity(nr_tick_around):
       analyzer.df_liquidity.columns = ['block', 'active_tick', 'tick', 'liquidity', 'price', 'amount0',
              'amount1', 'amount0_real', 'amount1_real']
       liquidity = pd.merge(analyzer.sampled_blocks,
              analyzer.df_liquidity,
              left_on='block_number',
              right_on='block')
       liquidity['active_tick_djusted'] = np.floor(liquidity['active_tick'] / pool.tick_spacing) * pool.tick_spacing
       liquidity['sb'] = liquidity['tick']-pool.tick_spacing*nr_tick_around
       liquidity['sa'] = liquidity['tick']+pool.tick_spacing*nr_tick_around
       liquidity.loc[liquidity['tick']<liquidity['active_tick'],'amount0_real'] = 0
       liquidity.loc[liquidity['tick']>liquidity['active_tick'],'amount1_real'] = 0
       active_liquidity = liquidity[(liquidity['active_tick_djusted'] >= liquidity['sb']) & (liquidity['active_tick_djusted'] <= liquidity['sa'])]
       return active_liquidity, liquidity

In [114]:
active_liquidity, liquidity = subselect_active_liquidity(3)

In [129]:
# why does it work for 0.3% pool but not 0.05?
# this is probably due to tick spacing being incorrect - check 
# also we need to adjust the volume we select ticks by in more liquid pool

In [115]:
active_liquidity.head(7)

,block_number,block_timestamp,cumulative_dollar_volume,datetime,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
18087,20654315,1725176735,1.023980e+06,2024-09-01 07:45:35,20654315,198182,198000,-1834013288,2519.992111,2.307628e+11,9.184796e+19,0.000000,91.847955,198180.0,197820,198180
18088,20654315,1725176735,1.023980e+06,2024-09-01 07:45:35,20654315,198182,198060,355223477,2504.918179,2.227682e+11,8.919952e+19,0.000000,89.199517,198180.0,197880,198240
18089,20654315,1725176735,1.023980e+06,2024-09-01 07:45:35,20654315,198182,198120,-426037808,2489.934415,2.209955e+11,8.902222e+19,0.000000,89.022215,198180.0,197940,198300
18090,20654315,1725176735,1.023980e+06,2024-09-01 07:45:35,20654315,198182,198180,110264033,2475.040281,2.129395e+11,2.971465e+18,0.000000,2.971465,198180.0,198000,198360
18091,20654315,1725176735,1.023980e+06,2024-09-01 07:45:35,20654315,198182,198240,847184650,2460.235239,1.568036e+11,6.392668e+19,156803.568710,0.000000,198180.0,198060,198420
18092,20654315,1725176735,1.023980e+06,2024-09-01 07:45:35,20654315,198182,198300,1740439499,2445.518757,1.481769e+11,6.077325e+19,148176.935541,0.000000,198180.0,198120,198480
18093,20654315,1725176735,1.023980e+06,2024-09-01 07:45:35,20654315,198182,198360,1456892502,2430.890305,1.373859e+11,5.668651e+19,137385.929321,0.000000,198180.0,198180,198540


In [101]:
active_liquidity['active_price'] = np.float64(active_liquidity['active_tick'].apply(lambda x: pool.adjust_price(pool.tick_to_price(x))))
active_liquidity['active_price_inverted'] = 1/active_liquidity['active_price']
active_liquidity['amount_locked'] = active_liquidity['amount1_real']*active_liquidity['active_price_inverted'] + active_liquidity['amount0_real']

/var/folders/11/llb_31fx359c3jzc04q5rklh0000gn/T/ipykernel_97895/3445529389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_liquidity['active_price_inverted'] = 1/active_liquidity['active_price']


/var/folders/11/llb_31fx359c3jzc04q5rklh0000gn/T/ipykernel_97895/3662684558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_liquidity['amount_locked'] = active_liquidity['amount1_real']*active_liquidity['active_price_inverted'] + active_liquidity['amount0_real']


In [100]:
active_liquidity

,block_number,block_timestamp,cumulative_dollar_volume,datetime,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa,active_price,amount_locked
18085,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,197830,-1.108687e+19,2563.196009,-1.681312e+12,-6.579143e+20,0.000000e+00,-657.914262,198000.0,197650,198010,0.000399,-2.623865e-01
18086,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,197890,-1.115034e+19,2547.863643,-1.685872e+12,-6.636686e+20,0.000000e+00,-663.668597,198000.0,197710,198070,0.000399,-2.646814e-01
18087,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,197950,-1.114526e+19,2532.622991,-1.680056e+12,-6.653592e+20,0.000000e+00,-665.359158,198000.0,197770,198130,0.000399,-2.653556e-01
18088,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,198010,-1.115483e+19,2517.473504,-1.676462e+12,-6.679310e+20,0.000000e+00,-667.930964,198000.0,197830,198190,0.000399,-2.663813e-01
18089,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,198070,-1.137317e+19,2502.414638,-1.704156e+12,-6.830506e+20,-1.704156e+06,0.000000,198000.0,197890,198250,0.000399,-1.704156e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7234872,20666284,1725321083,2.549669e+08,2024-09-02 23:51:23,20666284,197936,197770,-1.108147e+19,2578.620642,-1.685541e+12,-6.556237e+20,0.000000e+00,-655.623721,197880.0,197590,197950,0.000394,-2.585093e-01
7234873,20666284,1725321083,2.549669e+08,2024-09-02 23:51:23,20666284,197936,197830,-1.101984e+19,2563.196009,-1.671146e+12,-6.539364e+20,0.000000e+00,-653.936354,197880.0,197650,198010,0.000394,-2.578440e-01
7234874,20666284,1725321083,2.549669e+08,2024-09-02 23:51:23,20666284,197936,197890,-1.108192e+19,2547.863643,-1.675526e+12,-6.595958e+20,0.000000e+00,-659.595770,197880.0,197710,198070,0.000394,-2.600754e-01
7234875,20666284,1725321083,2.549669e+08,2024-09-02 23:51:23,20666284,197936,197950,-1.108079e+19,2532.622991,-1.670338e+12,-6.615105e+20,-1.670338e+06,0.000000,197880.0,197770,198130,0.000394,-1.670338e+06


In [ ]:
block['amount_locked'] = block['amount1_real'] * \
                         block.loc[block['active_tick_djusted']==block['tick']].price.values[0] \
                        + block['amount0_real']

In [88]:
active_liquidity.head(10)

,block_number,block_timestamp,cumulative_dollar_volume,datetime,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
18085,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,197830,-1.108687e+19,2563.196009,-1.681312e+12,-6.579143e+20,0.000000e+00,-657.914262,198000.0,197650,198010
18086,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,197890,-1.115034e+19,2547.863643,-1.685872e+12,-6.636686e+20,0.000000e+00,-663.668597,198000.0,197710,198070
18087,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,197950,-1.114526e+19,2532.622991,-1.680056e+12,-6.653592e+20,0.000000e+00,-665.359158,198000.0,197770,198130
18088,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,198010,-1.115483e+19,2517.473504,-1.676462e+12,-6.679310e+20,0.000000e+00,-667.930964,198000.0,197830,198190
18089,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,198070,-1.137317e+19,2502.414638,-1.704156e+12,-6.830506e+20,-1.704156e+06,0.000000,198000.0,197890,198250
18090,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,198130,-1.137758e+19,2487.445850,-1.699711e+12,-6.853687e+20,-1.699711e+06,0.000000,198000.0,197950,198310
47662,20652292,1725152399,2.088512e+06,2024-09-01 00:59:59,20652292,198054,197830,-1.108687e+19,2563.196009,-1.681312e+12,-6.579143e+20,0.000000e+00,-657.914262,198000.0,197650,198010
47663,20652292,1725152399,2.088512e+06,2024-09-01 00:59:59,20652292,198054,197890,-1.115034e+19,2547.863643,-1.685872e+12,-6.636686e+20,0.000000e+00,-663.668597,198000.0,197710,198070
47664,20652292,1725152399,2.088512e+06,2024-09-01 00:59:59,20652292,198054,197950,-1.114526e+19,2532.622991,-1.680056e+12,-6.653592e+20,0.000000e+00,-665.359158,198000.0,197770,198130
47665,20652292,1725152399,2.088512e+06,2024-09-01 00:59:59,20652292,198054,198010,-1.115483e+19,2517.473504,-1.676462e+12,-6.679310e+20,0.000000e+00,-667.930964,198000.0,197830,198190


In [78]:
liquidity.groupby(by='block_number')

,block_number,block_timestamp,cumulative_dollar_volume,datetime,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
0,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,-887270,2.854416e+15,3.401887e+50,1.576978e+32,4.650000e-07,0.000000e+00,0.0,198000.0,-887570,-886970
1,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,-887210,2.854416e+15,3.381538e+50,1.572254e+32,4.663000e-07,0.000000e+00,0.0,198000.0,-887510,-886910
2,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,-887150,2.854416e+15,3.361311e+50,1.567545e+32,4.678000e-07,0.000000e+00,0.0,198000.0,-887450,-886850
3,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,-887090,2.854416e+15,3.341204e+50,1.562849e+32,4.692000e-07,0.000000e+00,0.0,198000.0,-887390,-886790
4,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,-887030,2.854416e+15,3.321218e+50,1.558168e+32,4.706000e-07,0.000000e+00,0.0,198000.0,-887330,-886730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7246360,20666284,1725321083,2.549669e+08,2024-09-02 23:51:23,20666284,197936,887050,-1.064650e+19,0.000000e+00,-1.748124e-03,-5.835002e+35,-1.700000e-09,0.0,197880.0,886750,887350
7246361,20666284,1725321083,2.549669e+08,2024-09-02 23:51:23,20666284,197936,887110,-1.064650e+19,0.000000e+00,-1.742888e-03,-5.852532e+35,-1.700000e-09,0.0,197880.0,886810,887410
7246362,20666284,1725321083,2.549669e+08,2024-09-02 23:51:23,20666284,197936,887170,-1.064650e+19,0.000000e+00,-1.737667e-03,-5.870115e+35,-1.700000e-09,0.0,197880.0,886870,887470
7246363,20666284,1725321083,2.549669e+08,2024-09-02 23:51:23,20666284,197936,887230,-1.064650e+19,0.000000e+00,-1.732462e-03,-5.887751e+35,-1.700000e-09,0.0,197880.0,886930,887530


In [75]:
liquidity.head(10)

,block_number,block_timestamp,cumulative_dollar_volume,datetime,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
18083,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,197710,-1.114850e+19,2594.138096,-1.700832e+12,-6.576140e+20,0.000000e+00,-657.614008,198000.0,197410,198010
18084,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,197770,-1.114850e+19,2578.620642,-1.695737e+12,-6.595897e+20,0.000000e+00,-659.589714,198000.0,197470,198070
18085,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,197830,-1.108687e+19,2563.196009,-1.681312e+12,-6.579143e+20,0.000000e+00,-657.914262,198000.0,197530,198130
18086,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,197890,-1.115034e+19,2547.863643,-1.685872e+12,-6.636686e+20,0.000000e+00,-663.668597,198000.0,197590,198190
18087,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,197950,-1.114526e+19,2532.622991,-1.680056e+12,-6.653592e+20,0.000000e+00,-665.359158,198000.0,197650,198250
18088,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,198010,-1.115483e+19,2517.473504,-1.676462e+12,-6.679310e+20,0.000000e+00,-667.930964,198000.0,197710,198310
18089,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,198070,-1.137317e+19,2502.414638,-1.704156e+12,-6.830506e+20,-1.704156e+06,0.000000,198000.0,197770,198370
18090,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,198130,-1.137758e+19,2487.445850,-1.699711e+12,-6.853687e+20,-1.699711e+06,0.000000,198000.0,197830,198430
18091,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,198190,-1.137704e+19,2472.566601,-1.694539e+12,-6.873950e+20,-1.694539e+06,0.000000,198000.0,197890,198490
18092,20652150,1725150683,1.087745e+06,2024-09-01 00:31:23,20652150,198050,198250,-1.137704e+19,2457.776356,-1.689463e+12,-6.894602e+20,-1.689463e+06,0.000000,198000.0,197950,198550


In [42]:
df.head(2)

,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
0,20430379,195527,-887220,1.640134e+15,3.384921e+50,9.038614e+31,2.678000e-07,9.038614e+25,0.0,195480.0,-887520,-886920
1,20430379,195527,-887160,1.715982e+15,3.364674e+50,9.428278e+31,2.811000e-07,9.428278e+25,0.0,195480.0,-887460,-886860


In [48]:
active_df.tail(15)

,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
18104,20728120,198911,199020,5.468189e+08,2275.638951,1.408453e+11,6.207858e+19,140845.296202,62.078585,198900.0,198720,199320
18105,20728120,198911,199080,8.444734e+08,2262.026676,1.349929e+11,5.985714e+19,134992.871194,59.857136,198900.0,198780,199380
18106,20728120,198911,199140,1.239980e+09,2248.495825,1.161149e+11,5.179629e+19,116114.886906,51.796288,198900.0,198840,199440
18107,20728120,198911,199200,-1.205872e+09,2235.045913,1.083920e+11,4.864223e+19,108391.969226,48.642230,198900.0,198900,199500
18093,20729203,198693,198360,2.105499e+09,2430.890305,1.639856e+11,6.766173e+19,163985.580815,67.661733,198660.0,198060,198660
18094,20729203,198693,198420,-1.774291e+08,2416.349356,1.638760e+11,6.802340e+19,163875.961490,68.023402,198660.0,198120,198720
18095,20729203,198693,198480,2.472936e+08,2401.895388,1.666370e+11,6.958571e+19,166636.963126,69.585712,198660.0,198180,198780
18096,20729203,198693,198540,3.363394e+07,2387.527880,1.656189e+11,6.957677e+19,165618.900462,69.576770,198660.0,198240,198840
18097,20729203,198693,198600,-6.330146e+08,2373.246314,1.652350e+11,6.983323e+19,165235.035006,69.833232,198660.0,198300,198900
18098,20729203,198693,198660,-8.036493e+07,2359.050177,7.677805e+10,3.731111e+19,76778.053549,37.311106,198660.0,198360,198960


In [50]:
block = df.loc[df['block']==20729203]

In [51]:
block

,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
0,20729203,198693,-887220,-3.963300e+08,3.384921e+50,1.444653e+32,4.281000e-07,1.444653e+26,0.000000e+00,198660.0,-887520,-886920
1,20729203,198693,-887160,-1.775695e+09,3.364674e+50,1.481999e+32,4.418000e-07,1.481999e+26,0.000000e+00,198660.0,-887460,-886860
2,20729203,198693,-887100,-1.775695e+09,3.344547e+50,1.477560e+32,4.431000e-07,1.477560e+26,0.000000e+00,198660.0,-887400,-886800
3,20729203,198693,-887040,-1.775695e+09,3.324541e+50,1.473134e+32,4.444000e-07,1.473134e+26,0.000000e+00,198660.0,-887340,-886740
4,20729203,198693,-886980,-1.775695e+09,3.304654e+50,1.468722e+32,4.458000e-07,1.468722e+26,0.000000e+00,198660.0,-887280,-886680
...,...,...,...,...,...,...,...,...,...,...,...,...
29570,20729203,198693,886980,-1.253468e+08,0.000000e+00,3.965000e-07,1.314333e+32,0.000000e+00,1.314333e+14,198660.0,886680,887280
29571,20729203,198693,887040,-1.253468e+08,0.000000e+00,3.953000e-07,1.318282e+32,0.000000e+00,1.318282e+14,198660.0,886740,887340
29572,20729203,198693,887100,-1.253468e+08,0.000000e+00,3.942000e-07,1.322242e+32,0.000000e+00,1.322242e+14,198660.0,886800,887400
29573,20729203,198693,887160,-1.253468e+08,0.000000e+00,3.930000e-07,1.326215e+32,0.000000e+00,1.326215e+14,198660.0,886860,887460


In [56]:
block.loc[block['tick']<block['active_tick'],'amount0_real'] = 0
block.loc[block['tick']>block['active_tick'],'amount1_real'] = 0

In [57]:
block.loc[(block['price']>3200)&(block['price']<3500)]

,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_djusted,sb,sa
18033,20729203,198693,194760,8.001877e+08,3484.203865,1.284673e+12,3.698212e+20,0.0,369.821195,198660.0,194460,195060
18034,20729203,198693,194820,1.444285e+09,3463.362272,1.270196e+12,3.678541e+20,0.0,367.854142,198660.0,194520,195120
18035,20729203,198693,194880,-6.502686e+08,3442.645347,1.286435e+12,3.747989e+20,0.0,374.798853,198660.0,194580,195180
18036,20729203,198693,194940,1.243972e+09,3422.052345,1.282127e+12,3.757917e+20,0.0,375.791716,198660.0,194640,195240
18037,20729203,198693,195000,-8.999785e+07,3401.582525,1.316290e+12,3.881265e+20,0.0,388.126481,198660.0,194700,195300
18038,20729203,198693,195060,9.852590e+08,3381.235151,1.278661e+12,3.792999e+20,0.0,379.299899,198660.0,194760,195360
18039,20729203,198693,195120,-3.182985e+07,3361.009489,1.275292e+12,3.805773e+20,0.0,380.577278,198660.0,194820,195420
18040,20729203,198693,195180,-1.891584e+09,3340.904811,1.262271e+12,3.789582e+20,0.0,378.958250,198660.0,194880,195480
18041,20729203,198693,195240,-9.643397e+08,3320.920395,1.258694e+12,3.801584e+20,0.0,380.158381,198660.0,194940,195540
18042,20729203,198693,195300,1.967965e+09,3301.055520,1.258491e+12,3.823844e+20,0.0,382.384400,198660.0,195000,195600


In [65]:
block.loc[block['active_tick_djusted']==block['tick']].price.values[0]

np.float64(2359.0501771851)

In [68]:
block['amount_locked'] = block['amount1_real'] * \
                         block.loc[block['active_tick_djusted']==block['tick']].price.values[0] \
                        + block['amount0_real']

/var/folders/11/llb_31fx359c3jzc04q5rklh0000gn/T/ipykernel_90815/101492585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  block['amount_locked'] = block['amount1_real']*block.loc[block['active_tick_djusted']==block['tick']].price.values[0] + block['amount0_real']


In [90]:
block

NameError: name 'block' is not defined

In [69]:
block['amount_locked'].sum()

np.float64(60145883.01865217)

In [93]:
current_tick = int(data["current_pool_tick"])
active_tick = (
    math.floor(current_tick / pool.tick_spacing) * pool.tick_spacing
)
sb = active_tick-pool.tick_spacing*10
sa = active_tick+pool.tick_spacing*10



In [99]:
df['amount0_real'].sum()

np.float64(4.9621046174146895e+28)

In [100]:
df['amount1_real'].sum()

np.float64(4.427571698413304e+16)